## Roll number: 2023122002, 2023121013, 2023121006


### Instructions
 * Fill in the roll-number in the cell above.
 * Code must be submitted in Python in jupyter notebooks. We highly recommend using anaconda/miniconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 * For this assignment, you will be using Open3D  extensively. Refer to [Open3D](http://www.open3d.org/docs/release/) documentation.
 *  Most of the questions require you to **code your own functions** unless there is a need to call in the abilities of the mentioned libraries, such as Visualisation from Open3D. Make sure your code is modular since you will be reusing them for future assignments. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 *  All the representations are expected to be in a right-hand coordinate system.
<!--  * Answer to the descriptive questions should be answered in your own words. Copy-paste answers will lead to penalty. -->
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.
 * Commit the notebooks in the repo and any other results files under the result folder in the GitHub Classroom repo. 
 * Commits past the deadline will not be considered.
 * This is a group assignment. Discussions are encouraged but any sharing of code among different teams will be penalized. 

### Instructions for group formation
 * We have circulated google sheet in moodle to fill in team members. Please finalize the teams formation by 18th Aug (tentative deadline). Same teams will be working towards project and other 2 Assignments as well. 
 


# Q1: Transformations and Projections on Autonomous Driving Dataset (20 Points)

In this question, you will work with real world autonomous driving dataset (sequence in Waymo dataset). The dataset has LiDAR point clouds, images. You are required to demonstrate: 

**I. Various transformations of rotation matrices as described in below tasks.**

**II. Visualization as a result of above transformations in Open3D**

## Given data:

1.) `LiDAR Point Clouds` : Stored at each timestep in the folder `lidar`. The point clouds are provided in the ego frame attached to lidar sensor (vehicle's reference frame).

2.) `Images` : Stored at each timestep in the folder `images`. 

**Naming Convention** : {timestep}_{cam_no}.jpg where timestep is specified in 3 digits and cam_no : [0, 1, 2] indicates centre, left and right camera respectively.

3.) `Camera-to-Ego Transformations`: Stored in the folder `cam2ego`, which converts points from each camera's reference frame to the vehicle's (or ego) reference frame.

4.) `Ego-to-World Transformations`: Stored in the folder `ego2world`, which converts points from the vehicle's reference frame to the world frame W.

5.) `Camera Intrinsics`: Stored in the folder `intrinsics` provided for 3 cameras.




### Helper functions to read lidar data and camera instrinsics are provided below

In [1]:
# Imports

from copy import copy
import os
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from scipy.optimize import fsolve

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Helper functions

def read_transformation(filepath):
    data = np.loadtxt(filepath).reshape(4,4)
    matrix = data.reshape(4, 4)
    return matrix

def read_cam2ego_transformation(camera):
    return read_transformation(f'cam2ego/{camera}.txt')

def read_ego2world_transformation(timestep):
    return read_transformation(f'ego2world/{timestep:03d}.txt')

def read_image(timestep, camera):
    image = cv2.imread(f'images/{timestep:03d}_{camera}.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def display_image(image, title=''):
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    plt.show()

def read_intrinsic(camera):
    intrinsic = np.loadtxt(f'intrinsics/{camera}.txt')
    fx, fy, cx, cy = intrinsic[0], intrinsic[1], intrinsic[2], intrinsic[3]
    intrinsic_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    return intrinsic_matrix

def read_lidar_data(timestep):
    lidar_data = np.memmap(f'lidar/{timestep:03d}.bin',
                    dtype=np.float32,
                    mode="r",
                ).reshape(-1, 14)   # (165454, 14)

    lidar_origins = lidar_data[:, :3]
    lidar_points = lidar_data[:, 3:6]   # (165454, 3)
    lidar_ids = lidar_data[:, -1]   # (165454,)
    return lidar_origins, lidar_points, lidar_ids


**Note:** Even though Waymo dataset has 5 cameras, you are given the dataset corresponding to middle 3 cameras only. Please ignore other 2 cameras.

## Notation for tasks:

a.) `Global Reference Frame G`: Defined as the first ego frame (i.e., the translation vector of ego2world[0] is the origin of frame G in world frame W).

World Frame W: A fixed world reference frame.

b.) `Ego Frame`: Attached to the LiDAR and changes as the vehicle moves.

c.) `Camera Frames`: Each of the 5 cameras has its own frame, which changes as the vehicle moves.

Note: Axis directions of `Ego Frame` and `Camera Frames` are aligned with the Waymo Coordinate System (LiDAR) described below

## Coordinate Systems:

**OpenCV Coordinate System:** x right, y down, z front.

**Waymo Coordinate System (LiDAR):** x front, y left, z up.


![Waymo Setup](./waymo_setup.jpg "Waymo Setup")

Link to dataset (one sequence) : https://drive.google.com/drive/folders/17YDx2Yn1KmPjmlaHsoFz4Jpa8zpgovO2?usp=drive_link

If you want to try on other sequences as well, please refer to : https://waymo.com/open/

### `Task 1`. Transformations of LiDAR Point Clouds (10 points)

**Instructions:** 

Transform the LiDAR point clouds at all timesteps to the global reference frame G. Concatenate these transformed point clouds.
    
Visualization: Use Open3D to visualize the concatenated point cloud in the global reference frame G. Also, display the concatenation process at every timestep starting from first point cloud





In [ ]:
def apply_transformation(points, transformation):
    rotation = transformation[:3, :3]
    translation = transformation[:3, 3]
    transformed_points = (points @ rotation) + translation
    return transformed_points

def visualize_points(points, point_cloud):
    if point_cloud is None:
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(points)
        vis.add_geometry(point_cloud)
    else:
        point_cloud.points = o3d.utility.Vector3dVector(points)
        vis.update_geometry(point_cloud)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)
    return point_cloud

vis = o3d.visualization.Visualizer()
vis.create_window()
point_cloud = None

num_timesteps = 10
all_points = np.empty((0, 3))

for timestep in range(num_timesteps):
    _, points, _ = read_lidar_data(timestep)
    transformation = read_ego2world_transformation(timestep)
    transformed_points = apply_transformation(points, transformation)
    all_points = np.concatenate([all_points, transformed_points])
    point_cloud = visualize_points(all_points, point_cloud)

vis.run()
vis.destroy_window()

### `Task 2`. Projecting LiDAR Point Clouds onto images (10 points)

**Instructions:**

Transform the concatenated point cloud from task 1 to the frame of each of the 3 cameras at timesteps `0, 20, and 55`. 
    
Project these transformed point clouds onto the respective camera frames using the provided camera intrinsics. Concatenated point cloud would be very dense, so randomly select arbitrary number of points for better visualization. 

**Projected image pixel x : K * X_3d where X_3d is the 3d point in camera frame.**
    
Visualization: Overlay the projected points onto the camera images and visualize them.

**For example:** Overlayed concatenated point cloud on camera `000_0.png` and `030_2.png` are shown below

<table><tr>
<td> <img src="./projected_000_0.png" alt="Drawing" style="width: 750px;"/> </td>
<td> <img src="./projected_030_2.png" alt="Drawing" style="width: 750px;"/> </td>
</tr></table>



In [ ]:
##############################################################################
# TODO: TASK 2
##############################################################################

def apply_inverse_transformation(points, transformation):
    rotation = transformation[:3, :3]
    translation = transformation[:3, 3]
    transformed_points = (points - translation) @ (rotation.T)
    return transformed_points

def image_projection(points, intrinsic, image_height, image_width):
    image_points = points @ intrinsic.T
    image_points[:, :2] /= np.where(image_points[:, [2]] != 0, image_points[:, [2]], 1)
    image_points = np.rint(image_points).astype(int)
    in_range_mask = (
        (0 <= image_points[:, 0]) & (image_points[:, 0] < image_height) &
        (0 <= image_points[:, 1]) & (image_points[:, 1] < image_width) &
        (0 <= image_points[:, 2])
    )
    image_points_masked = image_points[in_range_mask]
    return image_points_masked

def visualize_projection(image, image_points, sample_fraction=0.05, marker_size=0.2):
    plt.figure(figsize=(8,6))
    sample_size = int(sample_fraction * image_points.shape[0])
    sample_indices = np.random.choice(image_points.shape[0], size=sample_size, replace=False)
    sampled_image_points = image_points[sample_indices]
    plt.imshow(image)
    plt.axis('off')
    plt.scatter(sampled_image_points[:, 1], sampled_image_points[:, 0], c='lime', s=marker_size)
    plt.show()

timestep = 0
camera = 0

ego_transformation = read_ego2world_transformation(timestep)
all_points_transformed = apply_inverse_transformation(all_points, ego_transformation)

camera_transformation = read_cam2ego_transformation(camera)
all_points_transformed = apply_inverse_transformation(all_points_transformed, camera_transformation)

waymo_to_cv = np.array([
    [0.0, -1.0, 0.0, 0.0],
    [0.0, 0.0, -1.0, 0.0],
    [1.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]
], dtype=np.float32)
all_points_transformed = apply_transformation(all_points_transformed, waymo_to_cv)

intrinsic = read_intrinsic(camera)
image = read_image(timestep, camera)
image_points = image_projection(all_points_transformed, intrinsic, image.shape[0], image.shape[1])

visualize_projection(image, image_points)

## Bonus

### `Task 3`. Compute Depth Image from Projected Point Cloud in camera frame (5 points)

**Instructions:**

Using the projected point clouds to camera frame from task 2, visualize the depth image by considering only the z-coordinate of the projected points in the camera frame.

Visualization: Display the depth image for each of the 3 cameras at timesteps `0, 20, and 55` alongside the corresponding RGB image.

In [ ]:
##############################################################################
# TODO: TASK 3
##############################################################################

def apply_inverse_transformation(points, transformation):
    rotation = transformation[:3, :3]
    translation = transformation[:3, 3]
    transformed_points = (points - translation) @ (rotation.T)
    return transformed_points

def image_projection(points, intrinsic, image_height, image_width):
    image_points = points @ intrinsic.T
    image_points[:, :2] /= np.where(image_points[:, [2]] != 0, image_points[:, [2]], 1)
    image_points = np.rint(image_points).astype(int)
    in_range_mask = (
        (0 <= image_points[:, 0]) & (image_points[:, 0] < image_height) &
        (0 <= image_points[:, 1]) & (image_points[:, 1] < image_width) &
        (0 <= image_points[:, 2])
    )
    image_points_masked = image_points[in_range_mask]
    return image_points_masked

def visualize_projection(image, image_points, sample_fraction=0.05, marker_size=0.2):
    plt.figure(figsize=(8,6))
    sample_size = int(sample_fraction * image_points.shape[0])
    sample_indices = np.random.choice(image_points.shape[0], size=sample_size, replace=False)
    sampled_image_points = image_points[sample_indices]
    plt.imshow(image)
    plt.axis('off')
    plt.scatter(sampled_image_points[:, 1], sampled_image_points[:, 0], c=sampled_image_points[:, 2], s=marker_size, cmap='inferno')
    plt.show()

timestep = 0
camera = 0

ego_transformation = read_ego2world_transformation(timestep)
all_points_transformed = apply_inverse_transformation(all_points, ego_transformation)

camera_transformation = read_cam2ego_transformation(camera)
all_points_transformed = apply_inverse_transformation(all_points_transformed, camera_transformation)

waymo_to_cv = np.array([
    [0.0, -1.0, 0.0, 0.0],
    [0.0, 0.0, -1.0, 0.0],
    [1.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]
], dtype=np.float32)
all_points_transformed = apply_transformation(all_points_transformed, waymo_to_cv)

intrinsic = read_intrinsic(camera)
image = read_image(timestep, camera)
image_points = image_projection(all_points_transformed, intrinsic, image.shape[0], image.shape[1])

visualize_projection(image, image_points)

#### Note: You might be asked to show the above results for different timesteps and from one of the 3 cameras during evaluation/viva.

# Q2: Various Representations for Rotations and Gimbal lock (15 points)


#### 2.1 Euler angles (2.5 points)

a. Write a function that returns a rotation matrix given the angles (𝛼, 𝛽, 𝛾) = (2π/5, π/18, π/6) in radians (X-Y-Z). Do not use inbuilt functions.

b. Solve for angles using fsolve from scipy for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 &  0.944799  \\0.21984631 &  0.96542533  & 0.14007684 \\
   -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

In [3]:
##############################################################################
# TODO: Do tasks described in 2.1 (a)
##############################################################################

def rotation_X(alpha):
    return np.array([
        [1, 0, 0],
        [0, np.cos(alpha), -np.sin(alpha)],
        [0, np.sin(alpha),  np.cos(alpha)]
    ])

def rotation_Y(beta):
    return np.array([
        [ np.cos(beta), 0, np.sin(beta)],
        [0, 1, 0],
        [-np.sin(beta), 0, np.cos(beta)]
    ])

def rotation_Z(gamma):
    return np.array([
        [ np.cos(gamma), -np.sin(gamma), 0],
        [ np.sin(gamma),  np.cos(gamma), 0],
        [0, 0, 1]
    ])

def rotation_matrix(angles):
    alpha, beta, gamma = angles
    R = rotation_X(alpha) @ rotation_Y(beta) @ rotation_Z(gamma)
    return R

alpha = 2 * np.pi / 5
beta = np.pi / 18
gamma = np.pi / 6

R = rotation_matrix([alpha, beta, gamma])
print('Rotation Matrix:')
print(R)

Rotation Matrix:
[[ 0.85286853 -0.49240388  0.17364818]
 [ 0.29753193  0.18504195 -0.93660783]
 [ 0.42905713  0.85046922  0.30432233]]


In [ ]:
##############################################################################
# TODO: Do tasks described in 2.1 (b)
##############################################################################

def rotation_matrix_error(angles, target_matrix):
    R = rotation_matrix(angles)
    return np.abs(np.sum(R - target_matrix, axis=0))

M = np.array([
    [ 0.26200263, -0.19674724,  0.944799],
    [ 0.21984631,  0.96542533,  0.14007684],
    [-0.93969262,  0.17101007,  0.29619813]
])

initializations = [
    [0,                 0,              0],
    [np.pi / 6,     np.pi,  3 * np.pi / 2],
    [    np.pi, np.pi / 3,      np.pi / 4]
]

for angles in initializations:
    solution = fsolve(rotation_matrix_error, angles, args=(M,))
    error = rotation_matrix_error(solution, M).sum()
    solution %= 2 * np.pi
    # solution = np.degrees(solution)
    print(f'Initialization: {angles}')
    print(f'Solution: {solution}')
    print(f'Error: {error}')
    print()

#### 2.2 Equivalent angle–axis representation (2.5 points) 

 Write a function to convert equivalent angle–axis representation (with a general axis and angle) to matrix form and vice versa. \
Try it for $\theta = \pi/6$ and axis $K= [1, 2, 3]^T $

In [34]:
##############################################################################
# TODO: Do tasks described in 2.2 
##############################################################################

def angle_axis_to_matrix(angle, axis):
    # Normalize the axis
    axis = np.array(axis, dtype=np.float64)
    axis /= np.linalg.norm(axis)

    # Compute the skew-symmetric matrix
    K = np.array([
        [       0, -axis[2],  axis[1]],
        [ axis[2],        0, -axis[0]],
        [-axis[1],  axis[0],        0]
    ])

    # Rodrigues' rotation formula
    I = np.eye(3)
    sin_term = np.sin(angle) * K
    cos_term = (1 - np.cos(angle)) * (K @ K)
    R = I + sin_term + cos_term

    return R

def matrix_to_angle_axis(R):
    # Angle
    angle = np.arccos((np.trace(R) - 1) / 2)

    # When near 0 - no rotation
    if np.isclose(angle, 0):
        return 0, np.array([1, 0, 0])

    # When near 180 degrees
    elif np.isclose(angle, np.pi):
        eigenvalues, eigenvectors = np.linalg.eig(R)
        axis = eigenvectors[:, np.isclose(eigenvalues, 1)].real.flatten()
        return np.pi, axis

    # Finding the axis
    axis = (1 / (2 * np.sin(angle))) * np.array([
        R[2, 1] - R[1, 2],
        R[0, 2] - R[2, 0],
        R[1, 0] - R[0, 1]
    ])

    return angle, axis

# Initial values
angle = np.pi / 6
axis = np.array([1, 2, 3], dtype=np.float64)
axis /= np.linalg.norm(axis)
print("Original Angle-Axis representation:")
print(f"Angle: {angle}")
print(f"Axis: {axis}")

# Find rotation matrix
R = angle_axis_to_matrix(angle, axis)
print("\nRotation matrix:")
print(R)

# Find Angle-Axis representation
angle, axis = matrix_to_angle_axis(R)
print("\nConversion to Angle-Axis:")
print(f"Angle: {angle} radians")
print(f"Normalized Axis: {axis}")

Original Angle-Axis representation:
Angle: 0.5235987755982988
Axis: [0.26726124 0.53452248 0.80178373]

Rotation matrix:
[[ 0.87559502 -0.38175263  0.29597008]
 [ 0.42003109  0.90430386 -0.07621294]
 [-0.2385524   0.19104831  0.95215193]]

Conversion to Angle-Axis:
Angle: 0.5235987755982985 radians
Normalized Axis: [0.26726124 0.53452248 0.80178373]


#### 2.3 Gimbal lock (5 points)

Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given point cloud, data/toothless.ply. You have to show the above by animation (rotation along each axis one by one).

**Hint:** 
Create 3 disks perpendicular to each other representing axes for local frame of object. Show that in certain configuration, due to use of Euler angles we can lose a degree of freedom. 

Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by 20° around a particular axis, do so in increments of 5° 4 times to make it look like an animation.

In [82]:
##############################################################################
# TODO: Do tasks described in 2.3
##############################################################################

# Initialize torus
torus_x = o3d.geometry.TriangleMesh.create_torus(torus_radius=350.0, tube_radius=5.0)
torus_y = o3d.geometry.TriangleMesh.create_torus(torus_radius=325.0, tube_radius=5.0)
torus_z = o3d.geometry.TriangleMesh.create_torus(torus_radius=300.0, tube_radius=5.0)

# Assign colors to torus
torus_x.paint_uniform_color([1.0, 0.0, 0.0])
torus_y.paint_uniform_color([0.0, 1.0, 0.0])
torus_z.paint_uniform_color([0.0, 0.0, 1.0])

# Initialize axes, associated with torus
x_axis = np.array([1, 0, 0])
y_axis = np.array([0, 1, 0])
z_axis = np.array([0, 0, 1])

# Align torus with respective axes
torus_x.vertices = o3d.utility.Vector3dVector(np.asarray(torus_x.vertices) @ angle_axis_to_matrix(np.pi / 2, y_axis).T)
torus_y.vertices = o3d.utility.Vector3dVector(np.asarray(torus_y.vertices) @ angle_axis_to_matrix(np.pi / 2, x_axis).T)

# Initialize point cloud
point_cloud = o3d.io.read_point_cloud('toothless.ply')

# Initialize visualization
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(point_cloud)
vis.add_geometry(torus_x)
vis.add_geometry(torus_y)
vis.add_geometry(torus_z)

# Sequence of rotations
axes = [('y', 90), ('x', 45), ('z', 45)]

# Iterate over all rotation
for ax, degree in axes:

    # Rotate slowly until entire rotation complete
    angle = 0
    step = 5
    while angle < degree:
        angle += step

        # Compute rotation matrix
        if ax == "x":
            rotation = angle_axis_to_matrix(np.radians(step), x_axis)
        elif ax == "y":
            rotation = angle_axis_to_matrix(np.radians(step), y_axis)
        elif ax == "z":
            rotation = angle_axis_to_matrix(np.radians(step), z_axis)

        # Rotate the gimbals and point cloud
        if ax == 'x':
            torus_x.vertices = o3d.utility.Vector3dVector(np.asarray(torus_x.vertices) @ rotation.T)
            x_axis = x_axis @ rotation
        if ax == 'x' or ax == 'y':
            torus_y.vertices = o3d.utility.Vector3dVector(np.asarray(torus_y.vertices) @ rotation.T)
            y_axis = y_axis @ rotation
        torus_z.vertices = o3d.utility.Vector3dVector(np.asarray(torus_z.vertices) @ rotation.T)
        point_cloud.points = o3d.utility.Vector3dVector(np.asarray(point_cloud.points) @ rotation.T)
        z_axis = z_axis @ rotation

        # Update the visualizer
        vis.update_geometry(point_cloud)
        vis.update_geometry(torus_x)
        vis.update_geometry(torus_y)
        vis.update_geometry(torus_z)

        # Update renderer
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.1)

# Terminate
vis.run()
vis.destroy_window()

#### 2.4: Quaternions (5 points)

a. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.

b. Perform matrix multiplication of two 3×3 rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both cases is the same.

c. Try to interpolate any given model between two rotation matrices and visualize!

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.4 (a)
##############################################################################

def rotation_matrix_to_quaternions(R):
    trace = np.trace(R)

    if trace > 0:
        S = np.sqrt(trace + 1.0) * 2
        w = 0.25 * S
        x = (R[2, 1] - R[1, 2]) / S
        y = (R[0, 2] - R[2, 0]) / S
        z = (R[1, 0] - R[0, 1]) / S
    elif R[0, 0] > R[1, 1] and R[0, 0] > R[2, 2]:
        S = np.sqrt(1.0 + R[0, 0] - R[1, 1] - R[2, 2]) * 2
        w = (R[2, 1] - R[1, 2]) / S
        x = 0.25 * S
        y = (R[0, 1] + R[1, 0]) / S
        z = (R[0, 2] + R[2, 0]) / S
    elif R[1, 1] > R[2, 2]:
        S = np.sqrt(1.0 + R[1, 1] - R[0, 0] - R[2, 2]) * 2
        w = (R[0, 2] - R[2, 0]) / S
        x = (R[0, 1] + R[1, 0]) / S
        y = 0.25 * S
        z = (R[1, 2] + R[2, 1]) / S
    else:
        S = np.sqrt(1.0 + R[2, 2] - R[0, 0] - R[1, 1]) * 2
        w = (R[1, 0] - R[0, 1]) / S
        x = (R[0, 2] + R[2, 0]) / S
        y = (R[1, 2] + R[2, 1]) / S
        z = 0.25 * S

    return np.array([w, x, y, z])

def quaternions_to_rotation_matrix(q):
    qw, qx, qy, qz = q
    R = np.array([[1 - 2*qy**2 - 2*qz**2,   2*qx*qy - 2*qz*qw,          2*qx*qz + 2*qy*qw],
                  [2*qx*qy + 2*qz*qw,       1 - 2*qx**2 - 2*qz**2,      2*qy*qz - 2*qx*qw],
                  [2*qx*qz - 2*qy*qw,       2*qy*qz + 2*qx*qw,          1 - 2*qx**2 - 2*qy**2]])
    return R

# Initialization
R = np.array([[0.866, -0.5, 0],
              [0.5, 0.866, 0],
              [0, 0, 1]])

# Convert to Quaternion
q = rotation_matrix_to_quaternions(R)
print("Quaternion:", q)

# Convert to Rotation Matrix
R_converted = quaternions_to_rotation_matrix(q)
print("Rotation Matrix:\n", R_converted)

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.4 (b)
##############################################################################

def multiply_rotation_matrix(R1, R2):
    return R1 @ R2

def multiply_quaternions(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    w = w1*w2 - x1*x2 - y1*y2 - z1*z2
    x = w1*x2 + x1*w2 + y1*z2 - z1*y2
    y = w1*y2 - x1*z2 + y1*w2 + z1*x2
    z = w1*z2 + x1*y2 - y1*x2 + z1*w2
    return np.array([w, x, y, z])

# Initialize rotation matrix
R1 = np.array([[0.866, -0.5, 0],
               [0.5, 0.866, 0],
               [0, 0, 1]])
R2 = np.array([[1, 0, 0],
               [0, 0.866, -0.5],
               [0, 0.5, 0.866]])
RProduct = multiply_rotation_matrix(R1, R2)
print("Multiplication of Rotation Matrices:\n", RProduct)

# Convert to quaternion and perform multiplication
q1 = rotation_matrix_to_quaternions(R1)
q2 = rotation_matrix_to_quaternions(R2)
q_product = multiply_quaternions(q1, q2)
print("\nMultiplication of Quaternions:", q_product)

# Convert quaternion product back to rotation matrix
R_from_quaternion_product = quaternions_to_rotation_matrix(q_product)
print("\nRotation Matrix from multiplication of Quaternions: \n", R_from_quaternion_product)
print("\nDifference betwen Rotation Matrix and Quaternion Product: \n", RProduct -  R_from_quaternion_product)

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.4 (c)
##############################################################################

# Function to interpolate between two rotation matrices
def interpolate_rotation_matrix(R1, R2, t):
    return (1 - t) * R1 + t * R2

# First Rotation matrix
R1 = np.array([[0.866, -0.5, 0],
               [0.5, 0.866, 0],
               [0, 0, 1]])

# Second Rotation matrix
R2 = np.array([[1, 0, 0],
               [0, 0.866, -0.5],
               [0, 0.5, 0.866]])

# Initialize the visualization
pcd = o3d.io.read_point_cloud("toothless.ply")
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)

# Interpolate
t_values = np.linspace(0, 1, 50)
for t in t_values:
    R_interp = interpolate_rotation_matrix(R1, R2, t)
    pcd.rotate(R_interp, center=(0, 0, 0))
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)

# Terminate
vis.run()
vis.destroy_window()

# Q3: Interpolation between transformations (15 points)

Given 2 random transformation matrices, interpolate the given point cloud **toothless.ply** from `T2` to `T1` and visualize it.

We will use the `generateTransformation()` function to generate a random Transformation matrix. You can write your own `generateTransformation()` function for testing, but we will replace it with our own so make sure that your code works for general cases.

Ensure that your visualization shows the starting and ending configurations during interpolation.

Your final output should look something like this:
![Visualization](./out.gif)

In [ ]:
def generateTransformation():
    """ Generate a random 4x4 transformation matrix. """

    R = o3d.geometry.get_rotation_matrix_from_xyz(np.random.rand(3) * 2 * np.pi)
    T = np.random.rand(3) * 5000
    transformation = np.vstack((np.hstack((R, T.reshape(-1, 1))), [0, 0, 0, 1]))
    return transformation

In [ ]:
def interpolate_transformations(T1, T2, num_steps):
    """ Interpolate between two transformations. """

    interpolated_transforms = []
    for t in np.linspace(0, 1, num_steps):
        rot_1, trans_1 = T1[:3, :3], T1[:3, 3]
        rot_2, trans_2 = T2[:3, :3], T2[:3, 3]
        R = rot_1 * (1 - t) + rot_2 * t
        T = trans_1 * (1 - t) + trans_2 * t
        transform = np.vstack((np.hstack((R, T.reshape(-1, 1))), [0, 0, 0, 1]))
        interpolated_transforms.append(transform)
    return interpolated_transforms

def apply_transformation(pcd, transformation):
    """ Apply a transformation matrix to copy of a point cloud. """

    new_pcd = o3d.geometry.PointCloud()
    new_pcd.points = pcd.points
    new_pcd.colors = pcd.colors
    new_pcd = new_pcd.transform(transformation)
    return new_pcd

# Generate two random transformations
T1 = generateTransformation()
T2 = generateTransformation()

# Load the point cloud
point_cloud = o3d.io.read_point_cloud("toothless.ply")
original_points = copy(point_cloud.points)
print(type(original_points))

# Apply initial and final transformations
initial_point_cloud = apply_transformation(point_cloud, T1)
final_point_cloud = apply_transformation(point_cloud, T2)

# Create visualizer
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(point_cloud)
vis.add_geometry(initial_point_cloud)
vis.add_geometry(final_point_cloud)

# Interpolate between T1 and T2
num_steps = 50
interpolated_transforms = interpolate_transformations(T1, T2, num_steps)

# Visualize interpolation
for transform in interpolated_transforms:
    point_cloud.points = original_points
    point_cloud.transform(transform)
    vis.update_geometry(point_cloud)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)

# Terminate
vis.run()
vis.destroy_window()

## Bonus (5 points)

In [ ]:
##############################################################################
# TODO: Implement the above question using spherical linear interpolation (slerp)
##############################################################################

from scipy.spatial.transform import Rotation as R

def slerp(q1, q2, t):
    """ Perform spherical linear interpolation between two quaternions. """

    # Compute the dot product to determine the angle between the quaternions
    dot = np.dot(q1, q2)
    
    # Adjust for the case when the dot product is negative
    if dot < 0.0:
        q2 = -q2
        dot = -dot

    # Clamp dot to the range [0, 1] to avoid numerical issues
    dot = np.clip(dot, 0.0, 1.0)

    # Compute the interpolation coefficients
    theta_0 = np.arccos(dot)
    theta = theta_0 * t
    q2 = q2 - q1 * dot
    q2 = q2 / np.linalg.norm(q2)

    # Compute the interpolated quaternion
    q_interp = q1 * np.cos(theta) + q2 * np.sin(theta)
    q_interp = q_interp / np.linalg.norm(q_interp)

    return q_interp

def interpolate_transformations(T1, T2, num_steps):
    """ Interpolate between two transformations. """
    interpolated_transforms = []
    R1, T1_part = T1[:3, :3], T1[:3, 3]
    R2, T2_part = T2[:3, :3], T2[:3, 3]

    # Convert rotation matrices to quaternions
    q1 = R.from_matrix(R1).as_quat()
    q2 = R.from_matrix(R2).as_quat()

    for t in np.linspace(0, 1, num_steps):
        # Interpolate rotation quaternions using SLERP
        q_interp = slerp(q1, q2, t)
        R_interp = R.from_quat(q_interp).as_matrix()

        # Interpolate translations
        T_interp = T1_part * (1 - t) + T2_part * t

        # Combine rotation and translation into a 4x4 transformation matrix
        transform = np.vstack((np.hstack((R_interp, T_interp.reshape(-1, 1))), [0, 0, 0, 1]))
        interpolated_transforms.append(transform)
    
    return interpolated_transforms

def apply_transformation(pcd, transformation):
    """ Apply a transformation matrix to a point cloud. """

    new_pcd = o3d.geometry.PointCloud()
    new_pcd.points = pcd.points
    new_pcd.colors = pcd.colors
    new_pcd = new_pcd.transform(transformation)
    return new_pcd

# Generate two random transformations
T1 = generateTransformation()
T2 = generateTransformation()

# Load the point cloud
point_cloud = o3d.io.read_point_cloud("toothless.ply")
original_points = copy(point_cloud.points)
print(type(original_points))

# Apply initial and final transformations
initial_point_cloud = apply_transformation(point_cloud, T1)
final_point_cloud = apply_transformation(point_cloud, T2)

# Create visualizer
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(point_cloud)
vis.add_geometry(initial_point_cloud)
vis.add_geometry(final_point_cloud)

# Interpolate between T1 and T2
num_steps = 50
interpolated_transforms = interpolate_transformations(T1, T2, num_steps)

# Visualize interpolation
for transform in interpolated_transforms:
    point_cloud.points = original_points
    point_cloud.transform(transform)
    vis.update_geometry(point_cloud)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)

# Terminate
vis.run()
vis.destroy_window()

#### References: https://en.wikipedia.org/wiki/Slerp